In [25]:
%run '04_Homogeneous_Variety.ipynb'

In [26]:
class Equivariant_Vector_Bundle ( Algebraic_Structure ) :

    def __add__ ( self , \
                  other:"Irreducible_Equivariant_Vector_Bundle" or "Extension_Of_Equivariant_Vector_Bundles" or "Direct_Sum_Of_Equivariant_Vector_Bundles" \
                ) -> "Irreducible_Equivariant_Vector_Bundle" or "Extension_Of_Equivariant_Vector_Bundles" or "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the coproduct of two equivariant vector bundles."""
        if isinstance( other , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) :
            New_Constituent_Parts = self.Constituent_Parts() + other.Constituent_Parts()
            Sum = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=New_Constituent_Parts )
            Sum.Tidy_Up()

            if Sum.Is_Irreducible() : return Sum[0]
            else                    : return Sum

        else :
            raise ValueError('The input for ``other`` is inappropriate type.')


    def __call__ ( self , *Twists:tuple[int] ) -> "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" :
        """Returns the tensor product of ``self`` with X.calO( Twists )."""
        assert len( Twists ) == len( self.Base_Space().Excluded_Nodes() ) , \
               ValueError('The length of ``Twists`` must coincide with the number of excluded nodes, namley ' + str(len( self.Base_Space().Excluded_Nodes() )) + '.')
        return self * self.Base_Space().calO( *Twists )


    def __pow__ ( self , e:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the power of ``self`` with respect to ``Exponent``."""
        assert e in Integers() , \
               ValueError('The exponent must be an positive integer.')

        Product = self.Base_Space().calO()
        if   e <  0 : Factor = self.Dual()
        elif e == 0 : return Product
        else : Factor = self

        for Counter in [ 1 .. abs(e) ] :
            Product *= Factor

        return Product


    def __sub__ ( self , other ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        return self + (-other)


    # Synonym for the method ``Exterior_Power``
    def Alt ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )


    # Synonym for the method ``Exterior_Power``
    def Alternating_Power ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )


    def Base_Space ( self ) -> Irreducible_Homogeneous_Variety :
        """Returns the attribute ``Base_Space``."""
        return self._Base_Space


    @staticmethod
    def Constructor ( Base_Space:Irreducible_Homogeneous_Variety , \
                      Constituent_Parts:list \
                    ) -> "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" :

        def Get_Summand_From_2Tuple ( Tuple:tuple ) -> "Irreducible_Equvariant_Vector_Bundle" :
            assert len(Tuple) == 2 , \
                   ValueError('The algorithm can not extract a vector bundle and multiplicity from a tuple of length '+str(len(Tuple))+'; it expects a tuple of length 2.')

            if   Tuple[0] == 0  :
                VB = Base_Space.calO()

            elif Tuple[0] in Base_Space.Parent_Group().Cartan_Type().root_system().weight_space() :
                VB = Base_Space.Irreducible_Equivariant_Vector_Bundle( Tuple[0] )

            elif isinstance( Tuple[0] , Irreducible_Equivariant_Vector_Bundle ) :
                VB = Tuple[0]

            elif isinstance( Tuple[0] , Young_Diagram ) :
                assert Tuple[0].Height() <= Base_Space.Cartan_Degree() , \
                       'The Young diagram need to be in the range from 0 to '+Base_Space.Cartan_Degree()+'.'
                fw = Base_Space.Basis('fw')
                M = matrix( QQ , [ vector( QQ , fw[Node].to_ambient()) for Node in [ 1 .. Base_Space.Cartan_Degree() ] ] )
                if Base_Space.Cartan_Family() == 'A' : Base_Change = M.submatrix( 0 , 0 , Base_Space.Cartan_Degree() , Base_Space.Cartan_Degree() ).inverse()
                else                                 : Base_Change = M.inverse()
                StdRepresentation = vector( QQ , Tuple[0].Row_Widths() + (Base_Space.Cartan_Degree()-Tuple[0].Height())*[ 0 ] )
                fwRepresentation  = StdRepresentation * Base_Change
                Highest_Weight = sum(   [ Base_Space.Null_Weight() ]
                                      + [ Coefficient*fw[Node] for Node , Coefficient in enumerate( fwRepresentation , start=1 ) ]
                                    )
                VB = Base_Space.Irreducible_Equivariant_Vector_Bundle( Highest_Weight )

            else :
                raise ValueError('The algorithm can not extract vector bundle from the first entry.')

            if   Tuple[1] in [ None , 0 ]         : Mult = WCR(0)
            elif Tuple[1] in ZZ  and 0 < Tuple[1] : Mult = Tuple[1] * WCR(1)
            elif Tuple[1] in WCR                  : Mult = Tuple[1]
            else : raise ValueError('The algorithm can not extract multiplicity from the second entry.')

            return VB , Mult

        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``Base_Space`` need to be an irreducible homogeneous variety.')

        WCR = Base_Space.Parent_Group().Weyl_Character_Ring()

        assert isinstance( Constituent_Parts , list ) , \
               TypeError('The input for ``Constituent_Parts`` need to be a list.')


        Summands = []
        for Constituent_Part_Counter , Constituent_Part in enumerate( Constituent_Parts , start=1 ) :
            if   Constituent_Part in [ None ] : pass

            elif Constituent_Part in [ 0 ] :
                Summands += [ Get_Summand_From_2Tuple( ( 0 , 1 ) ) ]

            elif Constituent_Part in Base_Space.Parent_Group().Cartan_Type().root_system().weight_space() or isinstance( Constituent_Part , Irreducible_Equivariant_Vector_Bundle ) :
                Summands += [ Get_Summand_From_2Tuple( ( Constituent_Part , 1 ) ) ]

            elif Constituent_Part in Base_Space.Parent_Group().Weyl_Character_Ring() :
                Summands += [ Get_Summand_From_2Tuple( ( 0 , Constituent_Part ) ) ]

            elif isinstance( Constituent_Part , Young_Diagram ) :
                Summands += [ Get_Summand_From_2Tuple( ( Constituent_Part , 1 ) ) ]

            elif isinstance( Constituent_Part , tuple ) and len(Constituent_Part) == 2 :
                Summands += [ Get_Summand_From_2Tuple( Constituent_Part ) ]

            else :
                raise ValueError('The input for the '+str(Constituent_Part_Counter)+'-th entry is inappropriate.')

        Result = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=Base_Space , Summands=Summands )
        if Result.Is_Irreducible() : return Result[0]
        else                       : return Result


    def Degree ( self ) -> int or list[ int ] :
        """Returns the degree of ``self``."""
        Coefficients = self.Determinant_Bundle().Highest_Weight().coefficients()
        if   len(Coefficients) == 0 : return 0
        elif len(Coefficients) == 1 : return Coefficients[0]
        else : return Coefficients


    # Synonym for the method ``Determinant_Bundle``.
    def Det ( self ) -> "Equivariant_Vector_Bundle" :
        return self.Determinant_Bundle()


    def Determinant_Bundle ( self ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the determinant line bundle of self (= rk-th top exterior power of ``self``)."""
        return self.Exterior_Power( self.Rank() )


    def Euler_Characteristic ( self ) -> "Weyl_Character" :
        """Returns sum_{ p } (-1)^p H^p( ``self`` )."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Result = sum([ (-1)^Degree * Weyl_Character for Degree , Weyl_Character in self.Cohomology().items() ])
        if Result == 0 : return WCR(0)
        else           : return Result


    def Euler_Sum ( self , \
                    other:"Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" \
                  ) -> "Weyl_Character" :
        """Returns sum_{ p } (-1)^p EXT^p(``self``,``other``)."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Result = sum([ (-1)^Degree * Weyl_Character for Degree , Weyl_Character in self.EXT(other).items() ])
        if Result == 0 : return WCR(0)
        else           : return Result


    def EXT ( self , \
              other:"Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" , \
              *p:int \
            ) -> dict[ int : "Weyl_Character" ] or "Weyl_Character":
        """Returns EXT(^p)(``self``,``other``)."""

        if   isinstance( self , Irreducible_Equivariant_Vector_Bundle )    : E1 = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=[ self ] )
        elif isinstance( self , Direct_Sum_Of_Equivariant_Vector_Bundles ) : E1 = self
        elif isinstance( self , Extension_Of_Equivariant_Vector_Bundles )  : E1 = self.SemiSimplification()
        else : raise TypeError(  'The input for ``self`` must be an equivariant vector bundle over an irreducible homogeneous variety or' \
                               + 'an equivariant extension of those ones.')

        if   isinstance( other , Irreducible_Equivariant_Vector_Bundle )    : E2 = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=other.Base_Space() , Summands=[ other ] )
        elif isinstance( other , Direct_Sum_Of_Equivariant_Vector_Bundles ) : E2 = other
        elif isinstance( other , Extension_Of_Equivariant_Vector_Bundles )  : E2 = other.SemiSimplification()
        else : raise TypeError(  'The input for ``other`` must be an equivariant vector bundle over an irreducible homogeneous variety or' \
                               + 'an equivariant extension of those ones.')

        try :
            return ( E1.Dual() * E2 ).Cohomology( *p )
        except :
            raise ValueError( 'Not able to compute EXT(E1,E2) with\n'+'E1='+str(E1)+'\n'+'E2='+str(E2)+'.' )


    def Extend_Equivariantly_By ( self , \
                                  other:"Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" \
                                ) -> "Extension_Of_Equivariant_Vector_Bundles" or None :
        """Returns the equivariant extension of ``self`` by ``other``."""
        return Extension_Of_Equivariant_Vector_Bundles ( Base_Space=self.Base_Space() , Quotient=self , Subobject=other )


    def Global_Sections ( self ) -> "Weyl_Character" :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        Cohomology = self.Cohomology()
        if 0 in Cohomology.keys() :
            return Cohomology[0]
        else :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            return WCR(0)


    # Synonym for the method ``Cohomology``
    def H ( self , *p:int ) -> dict[ int : "Weyl_Character" ] :
        """Returns the cohomology."""
        return self.Cohomology( *p )


    # Synonym for the method ``Global_Sections``
    def H0 ( self  ) -> "Weyl_Character" :
        """Returns the global sections (i.e. the zero-th cohomology)."""
        return self.Global_Sections()


    def HOM ( self , other ) -> "Weyl_Character" :
        """Returns the morphisms form ``self`` to ``other``."""
        EXT_Space = self.EXT(other)
        if 0 in EXT_Space.keys() : return EXT_Space[0]
        else :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            return WCR(0)


    def Is_Exceptional ( self , Test_Numerically:bool=False ) -> bool :
        """Tests if ``self`` is (numerically) exceptional, i.e. EXT(self,self) = kk[0] or Euler_Sum(self,self) = kk respectively."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        if Test_Numerically == False :
            if self.EXT( self ) == dict({ 0 : WCR(1) }) : return True
            else : return False

        elif Test_Numerically == True :
            if self.Euler_Sum(self) == WCR(1) : return True
            else : return False

        else :
            raise TypeError('The input for ``Test_Numerically`` need to be boolean.')


    def Is_Numerically_Exceptional ( self ) -> bool :
        """Tests if ``self`` is numerically exceptional, i.e. Euler_Sum(self,self) = kk."""
        return self.Is_Exceptional( Test_Numerically=True )


    def Is_Numerically_SemiOrthogonal_To ( self , \
                                           others:"Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" or \
                                                  list[ "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" ] , \
                                         ) -> Iterator[ bool ] :
        """Tests if ``self`` is numerically semi-orthogonal to ``other``; i.e. Euler_Sum(self,other)=0"""
        for Result in self.Is_SemiOrthogonal_To( others=others , Test_Numerically=True ) :
            yield Result


    def Is_Line_Bundle ( self ) -> bool :
        """Tests if `self` is line bundle."""
        return self.Rank() == 1


    def Is_SelfDual ( self ) -> bool :
        """Tests if ``self`` is self-dual."""
        return self == self.Dual()


    def Is_SemiOrthogonal_To ( self , \
                               others:"Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" or \
                                      list[ "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" ] , \
                               Test_Numerically:bool=False \
                             ) -> Iterator[ bool ] :
        """Tests if ``self`` is (numerically) semi-orthogonal to ``other``; i.e. EXT(self,other) = 0 or Euler_Sum(self,other)=0 respectively"""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

        if isinstance( others , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) : others = [ others ]
        if isinstance( others , list ) :
            for other_Counter , other in enumerate( others , start=1 ) :
                assert isinstance( other , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) , \
                       TypeError('The '+str(other_Counter)+'-th entry is not an equivariant vector bundle.')

                if Test_Numerically == False :
                    if self.EXT( other ) == dict({}) : yield True
                    else : yield False

                elif Test_Numerically == True :
                    if self.Euler_Sum( other ) == WCR(0) : yield True
                    else : yield False

                else :
                    raise TypeError('The input for ``Test_Numerically`` need to be boolean.')

        else : raise TypeError('The input for ``others`` need to be a single equivariant vector bundle or a list of those ones.')


    def Is_Trivial ( self ) -> bool :
        """Tests if ``self`` is trivial."""
        return self.Rank() == 0


    def Maximal_Numerically_Exceptional_Orbit ( self , Twist=None ) -> "Lefschetz_Collection" :
        """Returns the maximal exceptional oribt of ``self``."""
        if Twist == None :
            if self.Base_Space().Parabolic_Subgroup().Is_Maximal() : Twist = self.Base_Space().calO(1)
            else : raise ValueError( 'For a non-maximal paralic subgroup, one need to clearify the twist explicitly.' )
        assert isinstance( Twist , Irreducible_Equivariant_Vector_Bundle ) , \
               TypeError('The input for ``Twist`` must be a vector bundle over an irreducible homogeneous variety.')
        assert Twist.Is_Line_Bundle() , \
               ValueError('The input for ``Twist`` need to be a line bundle.')

        Orbit_Length = 0
        if self.Is_Numerically_Exceptional() :
            Orbit_Length = 1
            Moving_Object = self
            Reached_End_Of_Numerically_Exceptional_Orbit = False
            while not Reached_End_Of_Numerically_Exceptional_Orbit :
                Moving_Object *= Twist
                for Moving_Object_Is_Numerically_SemiOrthogonal_To_Self in Moving_Object.Is_Numerically_SemiOrthogonal_To( self ) :
                    if Moving_Object_Is_Numerically_SemiOrthogonal_To_Self : Orbit_Length += 1
                    else : Reached_End_Of_Numerically_Exceptional_Orbit = True
            Support_Pattern = Orbit_Length * [ 1 ]

        else : Support_Pattern = 0 * [ 1 ]

        return Lefschetz_Collection( Base_Space=self.Base_Space() , Starting_Block=[ self ] , Twist=Twist , Support_Pattern=Support_Pattern )


    # CAN BE DELETED!
    #def Multiply_By ( self , External_Multiplicity:"Weyl_Character" ) -> "Equivariant_Vector_Bundle" :
    #    """Returns the tensor product of ``self`` with some multiplicity."""
    #    WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
    #
    #    if External_Multiplicity in ZZ and 0 <= External_Multiplicity : External_Multiplicity = External_Multiplicity * WCR(1)
    #
    #    assert isinstance( External_Multiplicity , type(WCR(0)) ) , \
    #           TypeError('The multiplicity need to be a Weyl character of ' + str(WCR) + '.')
    #
    #    Factor = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=[ ( self.Base_Space().calO() , External_Multiplicity ) ] )
    #    return Factor * self


    def Numerical_Left_Mutation ( self:"Equivariant_Vector_Bundle" , other:"Equivariant_Vector_Bundle" ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the numerical left mutation of ``other`` through ``self``."""
        assert isinstance( self , Equivariant_Vector_Bundle ) , \
               'The input for ``self`` need to be an equivariant vector bundle.'
        e = self.SemiSimplification()
        assert isinstance( other , Equivariant_Vector_Bundle ) , \
               'The input for ``self`` need to be an equivariant vector bundle.'
        v = other.SemiSimplification()
        return e * ( e.Euler_Sum(v) ) - v


    def Numerical_Right_Mutation ( self:"Equivariant_Vector_Bundle" , other:"Equivariant_Vector_Bundle" ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the numerical left mutation of ``other`` through ``self``."""
        assert isinstance( self , Equivariant_Vector_Bundle ) , \
               'The input for ``self`` need to be an equivariant vector bundle.'
        e = self.SemiSimplification()
        assert isinstance( other , Equivariant_Vector_Bundle ) , \
               'The input for ``self`` need to be an equivariant vector bundle.'
        v = other.SemiSimplification()
        return e * ( v.Euler_Sum(e) ) - v


    # Synonyms for the function ``__pow__``
    def T ( self , e:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the tensor power of ``self`` (cf. to the method ``__pow__``)"""
        return self^e


    # Synonyms for the function ``__pow__``
    def Tensor_Power ( self , e:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the tensor power of ``self`` (cf. to the method ``__pow__``)"""
        return self^e


    # Synonym for the method ``Symmetric_Power``
    def S ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``p``-th symmetric power of ``self``."""
        return self.Symmetric_Power( p )


    def Slope ( self ) -> int :
        """Returns the slope of ``self``."""
        Degree = self.Degree()
        Rank = self.Rank()
        assert Degree in QQ , 'The ``Degree`` must be a rational number.'
        if Rank == 0 : return +Infinity
        else : return Degree / Rank


    def Standard_Stability ( self ) -> int :
        """Returns the standard stability of ``self``."""
        Degree = self.Degree()
        Rank = self.Rank()
        assert Degree in QQ , 'The ``Degree`` must be a rational number.'
        return -1 * Degree + sqrt(-1) * Rank


    # Synonym for the method ``Exterior_Power``
    def Wedge ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``e``-th exterior power of ``self``."""
        return self.Exterior_Power( p )


In [29]:
class Irreducible_Equivariant_Vector_Bundle ( Irreducible_Algebraic_Structure , Equivariant_Vector_Bundle ) :

    def __contains__ ( self , other ) -> bool :
        if   isinstance( other , Irreducible_Equivariant_Vector_Bundle )    :
            if other == self : return True
            else             : return False
        elif isinstance( other , Direct_Sum_Of_Equivariant_Vector_Bundles ) :
            if other.Is_Irreducible() and other[0] == self : return True
            else                                           : return False
        else :
            return False


    def __init__( self , Base_Space:Irreducible_Homogeneous_Variety , Highest_Weight:"Weight" ) -> None :
        """
        Initialize a irreducible equivariant vector bundle over an irreducible homogeneous variety.

        INPUT:
        - ``BaseSpace`` -- Irreducible_Homogeneous_Variety;
        - ``Weights`` -- Highest weight space;

        OUTPUT: None.
        """
        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``Base_Space`` must be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space

        assert Highest_Weight in Base_Space.Parent_Group().Cartan_Type().root_system().weight_space() , \
               ValueError('The input for ``Highest_Weight`` must be an element of the weight space, namely '+str(self.Base_Space().Parent_Group().Cartan_Type().root_system().weight_space())+'.')
        self._Highest_Weight = Highest_Weight


    def __mul__ ( self , \
                  other:Integer or "Weyl_Character" or "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundle" or "Extension_Of_Equivariant_Vector_Bundles" \
                ) -> "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundle" :
        """Tensor product of two equivariant vector bundles"""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

        if   isinstance( other , Integer ) :
            return self * (other*WCR(1))

        elif isinstance( other , type(WCR(0)) ) :
            return self * Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=[ ( self.Base_Space().calO() , other ) ] )

        elif isinstance( other , Irreducible_Equivariant_Vector_Bundle ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            # Product of Twists, i.e. sum of the highest weights supported over the total twisting part
            Prd_Twist_HW =    self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() ) \
                           + other.Highest_Weight_Supported_Over( Nodes=other.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )

            # Product of Levi parts, i.e. product of the representations of highest weights supported over the total Levi part
            Prd_Levi_Rep =    self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse') \
                           * other.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse')

            # In the remaining part, we merge the separate products over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
            def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
                Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
                List = []
                for HighestWeight , Multiplicity in Weyl_Character :
                    Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                    List += [ ( Coroots , Multiplicity ) ]
                return List
            # A product of two highest weights (= irreducible presentations) gives
            # in general a list of highest weights (= reducible presentations)
            n = self.Base_Space().Parent_Group().Cartan_Degree()
            e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
                # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
                # to apply base changes use by multiplication with appropriate Cartan matrix.
            Cartan_Matrix_From_fw_To_mx = self.Base_Space().Cartan_Matrix( From='fw' , To='mx' )
            Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
            mx = self.Base_Space().Basis('mx')

            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            Summands = []
            for Prd_HW , Prd_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Prd_Levi_Rep ) :
                # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
                fw_Coefficients = vector( QQ , n*[ 0 ] )
                fw_Coefficients += sum([ Prd_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
                # 3. Transform the coefficients from basis C (fw's) to B (mixed)
                mx_Coefficients = Cartan_Matrix_From_fw_To_mx * fw_Coefficients
                # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
                Prd_Levi_HW = 0*mx[1]
                Prd_Levi_HW += sum( [  Coefficient*mx[Node] for Node , Coefficient in enumerate( list(mx_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

                Summands += [ ( self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Prd_Twist_HW+Prd_Levi_HW ) , Prd_Mult*WCR(1) ) ]

            Result = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=Summands )
            if Result.Is_Irreducible() : return Result[0]
            else                       : return Result

        elif isinstance( other , Direct_Sum_Of_Equivariant_Vector_Bundles ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            return self.SemiSimplification() * other

        elif isinstance( other , Extension_Of_Equivariant_Vector_Bundles ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            return other.__class__( Base_Space=other.Base_Space() , Quotient=self*other.Quotient() , Subobject=self*other.Subobject() )

        else :
            raise ValueError( 'The input for ``other`` must be an equivariant vector bundle over an irreducible homogeneous variety or' \
                              + 'the extension of those ones.'
                            )


    def __neg__ ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        Base_Space = self.Base_Space()
        WCR = Base_Space.Parent_Group().Weyl_Character_Ring()
        Summands = [ ( self , -WCR(1) ) ]
        return Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=Base_Space , Summands=Summands )


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Highest_Weight()


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        return 'VB(' + str( self.Highest_Weight() ) + ')'


    def Coefficients ( self , Basis_Type:str='mx' ) -> vector :
        """
        Return the coefficients of the weights with respect to the basis sr, mx or fw.
        (For more details on the different basis types, we refer to the method ``Cartan_Matrix`` of the class ``Parabolic_Subgroup_In_Irreducible_Cartan_Group``.)
        """
        if Basis_Type in self.Base_Space().BASIS_LABELLING['sr'] :
            return self.Base_Space().Cartan_Matrix( From='fw' , To='sr' ) * self.Coefficients('fw')

        elif Basis_Type in self.Base_Space().BASIS_LABELLING['mx'] :
            return self.Base_Space().Cartan_Matrix( From='fw' , To='mx' ) * self.Coefficients('fw')

        elif Basis_Type in self.Base_Space().BASIS_LABELLING['fw'] :
            Vector = vector( QQ , self.Base_Space().Parent_Group().Cartan_Degree() )
            for Node , Coefficient in self.Highest_Weight() : Vector[ Node-1 ] = Coefficient
            return Vector

        else: raise ValueError('The input for the variable ``basis`` is unknown.')


    def Cohomology ( self , *p:int ) -> dict[ int : "Weyl_Character" ] :
        """
        Returns the cohomology.

        INPUT:
        - ``self`` -- VectorBundle_Over_HomogeneousVariety; E over X=G/P.

        OUTPUT:
        - ``Output`` -- Dictionary; Cohomology = { Degree d : H^d( X , E ) }

        ALGORITHM:
        Compute the cohomology by Borel-Weil-Bott, namely Theorem 2.15. in [KP2016].

        Take any λ ∈ P^+_L⊂P_L = P_G (an element of the dominant cone).
        If λ+ρ is G-regular (i.e. does NOT ly on a wall of the Weyl chamber), ...
          ... then there exists a unique w∈WG such that w(λ+ρG) is dominant.
              In this case H^ℓ(w)(X,λ) = WCR( w(λ+ρ)−ρ ) and the other cohomology groups vanish.
        If λ+ρ is G-singular (i.e. lies on a wall of the Weyl chamber), ...
          ... then H^•(X,λ) = 0.

        REFERENCE:
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Rho = sum( self.Base_Space().Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weights() )
        Sum = self.Highest_Weight() + Rho

        Cohomology = {}
        if self.Base_Space().Is_Regular( Sum ) :
            [ Shifted_Sum , Direction ] = Sum.to_dominant_chamber(reduced_word=True)
            Degree = len(Direction)
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            Cohomology_Group = WCR( Shifted_Sum - Rho )
            Cohomology.update({ Degree : Cohomology_Group })

        if   len(p) == 0 : return Cohomology

        elif len(p) == 1 :
            if p in Cohomology.keys() : return Cohomology
            else : return dict({})

        else :
            raise ValueError('The input for ``p`` is of inappropriate length.')


    def Dual ( self ) -> "Irreducible_Equivaraint_Vector_Bundle" :
        """Returns the dual of ``self``."""
        # Dualize the highest weight supported over the twisting part (exluded nodes)
        Dual_Twist_HW = -1 * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() ) # Highest Weight supported over the total twisting part

        # Dualize the representation supported over the Levi part (included nodes)
        Dual_Levi_Rep = [ Weyl_Character.dual() for Weyl_Character in self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Fine') ]

        # In the remaining part, we merge the separate dualizations over both the twisting part (excluded nodes) and the Levi part (included nodes)
        # 1. Define subroutine to get coefficients of a given weyl character
        def Get_Coefficients_Of_Weyl_Character ( Weyl_Character ) :
            Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
            List = []
            for HighestWeight , Multiplicity in Weyl_Character :
                Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                List += Coroots
            return List
        # 2. Write the coefficients of the highest weight of the weyl character ``Dual_Levi_Rep`` in terms of the basis C (fw's)
        n = self.Base_Space().Parent_Group().Cartan_Degree()
        e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
            # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
            # to apply base changes use by multiplication with appropriate Cartan matrix.
        fw_Coefficients = vector( QQ , n*[ 0 ] )
        Relabellings = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Fine')
        for Component_Counter , Weyl_Character in enumerate( Dual_Levi_Rep ) :
            Relabelling = Relabellings[Component_Counter]
            Relabelling_Inverted = { Relabel : Node for Node , Relabel in Relabelling.items() }
            fw_Coefficients += sum([ Coefficient * e[ Relabelling_Inverted[Relabel] ]
                                     for Relabel , Coefficient in enumerate( Get_Coefficients_Of_Weyl_Character( Weyl_Character ) , start=1 )
                                   ])
        # 3. Transform the coefficients from basis C (fw's) to B (mixed)
        Cartan_Matrix_From_fw_To_mx = self.Base_Space().Cartan_Matrix( From='fw' , To='mx' )
        mx_Coefficients = Cartan_Matrix_From_fw_To_mx * fw_Coefficients
        # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
        mx = self.Base_Space().Basis('mx')
        Dual_Levi_HW = 0*mx[1]
        Dual_Levi_HW += sum( [  Coefficient*mx[Node] for Node , Coefficient in enumerate( list(mx_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])
        # 5. Finally merging is just adding the weights.
        Dual_HW = Dual_Twist_HW+Dual_Levi_HW

        return self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Dual_HW )


    def Exterior_Power ( self , p:int ) -> "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``e``-th exterior power of ``self``."""
        assert p in ZZ ,  \
               ValueError('The exponent ``p`` must be an integer.')

        if   p == 0 :
            return self.Base_Space().calO()

        elif p in [ 1 .. self.Rank() ] :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

            # ... the highest weight supported over the twisting part (excluded nodes)
            Ext_Twist_HW = p * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )
            # ... the highest weight supported over the Levi part (included nodes)
            Ext_Levi_Rep = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse').exterior_power(p)

            # In the remaining part, we merge the separate symmetric powers over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
            def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
                Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
                List = []
                for HighestWeight , Multiplicity in Weyl_Character :
                    Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                    List += [ ( Coroots , Multiplicity ) ]
                return List
            # A symmetric power of two highest weights (= irreducible presentations) gives
            # in general a list of highest weights (= reducible presentations)
            n = self.Base_Space().Parent_Group().Cartan_Degree()
            e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
                # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
                # to apply base changes use by multiplication with appropriate Cartan matrix.
            Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='fw' , To='mx' )
            Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
            mixed = self.Base_Space().Basis('mx')
            Summands = []
            for Ext_HW , Ext_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Ext_Levi_Rep ) :
                # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
                C_Coefficients = vector( QQ , n*[ 0 ] )
                C_Coefficients += sum([ Ext_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
                # 3. Transform the coefficients from basis C (fw's) to B (mixed)
                B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
                # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
                Ext_Levi_HW = 0*mixed[1]
                Ext_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

                Summands += [ ( self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Ext_Twist_HW+Ext_Levi_HW ) , Ext_Mult*WCR(1) ) ]

            Result = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=Summands )
            if Result.Is_Irreducible() : return Result[0]
            else                       : return Result

        else :
            return self.Base_Space().Zero_Vector_Bundle()


    def Highest_Weight ( self ) -> "Weight" :
        """Returns the attribute ``_Highest_Weight``."""
        return self._Highest_Weight


    def Highest_Weight_Supported_Over ( self , Nodes:set ) -> "Weight" :
        """Returns the parts of the highest weights supported over ."""
        All_Nodes = set( self.Base_Space().Parent_Group().Cartan_Type().index_set() )
        assert Nodes.issubset( All_Nodes ) , \
               ValueError('The input for ``Nodes`` must be a subset of all nodes ' + str( All_Nodes ) + '.')
        Coefficients = self.Coefficients('mx')
        mx = self.Base_Space().Basis('mx')
        return sum( [ Coefficients[Node-1] * mx[Node] for Node in Nodes ] )


    def Negative_Part ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        return self.Base_Space().Zero_Vector_Bundle()


    def Positive_Part ( self ) -> "Irreducible_Equivariant_Vector_Bundle" :
        return self


    def PullBack ( self , Image:Irreducible_Homogeneous_Variety , Dictionary:dict ) -> Equivariant_Vector_Bundle :
        """Returns the semi-simplification of the pull back of ``self`` to ``Image``."""

        Source = self.Base_Space()
        fw = Source.Basis('fw')
        Mapping = Source.Mapping( Image , Dictionary )

        #Pull back all weights and store the Levi-dominant ones.
        assert isinstance( Image , Irreducible_Homogeneous_Variety ) , \
               'The input for ``Image`` need Image be an irreducible homogeneous variety.'
        SemiSimplification = Image.Zero_Vector_Bundle()
        for Weight , Multiplicity in self.Weights() :
            PullBack_Weight = sum([ Image.Null_Weight() ] + [ Coefficient*Mapping[fw[Node]] for Node , Coefficient in Weight ])
            if Image.Is_Levi_Dominant( PullBack_Weight ) :
                SemiSimplification += Image.calU(PullBack_Weight) * Multiplicity

        #Get the highest weight and drop smaller weights.
        KeepRuning = True
        while KeepRuning :
            KeepRuning = False
            for IrdCmpnt1 in SemiSimplification.Irreducible_Components() :
                Weight1 = IrdCmpnt1.Highest_Weight()
                for Weight2 , Multiplicity2 in IrdCmpnt1.Weights() :
                    if Image.Is_Levi_Dominant( Weight2 ) :
                        if Image.Compare_Weights(Weight1,Weight2) == '>' :
                            IrdCmpnt2 = Image.calU(Weight2) * Multiplicity2
                            if IrdCmpnt2 in SemiSimplification :
                                SemiSimplification -= IrdCmpnt2
                                KeepRuning = True

        return SemiSimplification


    def Rank ( self ) -> int :
        """Returns the rank of ``self``."""
        return prod([ Representation.degree() for Representation in self.Representation_Supported_Over_Levi_Part( Decomposition_Style='Fine' ) ])


    def Representation_Supported_Over_Levi_Part ( self , Decomposition_Style:str='Coarse' ) -> "Weyl_Character" :
        Highest_Weight = self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Included_Nodes() ) # Highest Weight supported over the total Levi part
        Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style=Decomposition_Style)

        if   Decomposition_Style == 'Fine' :
            # Supported over the irreducible components of the Levi Part
            List = []
            for Component_Counter , Levi_Component in enumerate( self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Irreducible_Components() ) :
                WCR = Levi_Component.Weyl_Character_Ring()
                fw = Levi_Component.Cartan_Type().root_system().weight_space().fundamental_weights()
                List += [ WCR( sum([ Highest_Weight[Node]*fw[Relabel] for Node , Relabel in Relabelling[Component_Counter].items() ]) ) ]
            return List

        elif Decomposition_Style == 'Coarse' :
            # Supported over the total Levi Part
            WCR = self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Weyl_Character_Ring()
            Basis = self.Base_Space().Parabolic_Subgroup().Initialize_As_Cartan_Group().Cartan_Type().root_system().weight_space().fundamental_weights()
            return WCR( sum([ Highest_Weight[Node]*Basis[Relabel] for Node , Relabel in Relabelling.items() ]) )

        else : raise ValueError('The input for the decomposition style is unknown.')


    def SemiSimplification ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns ``self`` as direct sum."""
        return Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=[ self ] )


    def Support ( self ) -> list :
        """Returns the support of the highest weight."""
        return self.Highest_Weight().support()


    def Symmetric_Power ( self , p:int ) -> "Irreducible_Equivariant_Vector_Bundle" or "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the ``e``-th symmetric power of ``self``."""
        assert p in Integers() , \
               ValueError('The exponent ``e`` must be an integer.')

        if   p < 0 :
            return self.Base_Space().Zero_Vector_Bundle()

        elif p == 0 :
            return self.Base_Space().calO()

        else :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

            # ... the highest weight supported over the twisting part (excluded nodes)
            Sym_Twist_HW = p * self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )
            # ... the highest weight supported over the Levi part (included nodes)
            Sym_Levi_Rep = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse').symmetric_power(p)

            # In the remaining part, we merge the separate symmetric powers over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Define subroutine to get coefficients and multiplicities of a given weyl character
            def Get_Coefficients_And_Multiplicity_Of_Weyl_Character ( Weyl_Character ) :
                Simple_Coroots = WeylCharacterRing( Weyl_Character.cartan_type() ).simple_coroots()
                List = []
                for HighestWeight , Multiplicity in Weyl_Character :
                    Coroots = [ HighestWeight.inner_product( Element ) for Element in list(Simple_Coroots) ]
                    List += [ ( Coroots , Multiplicity ) ]
                return List
            # A symmetric power of two highest weights (= irreducible presentations) gives
            # in general a list of highest weights (= reducible presentations)
            n = self.Base_Space().Parent_Group().Cartan_Degree()
            e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
                # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
                # to apply base changes use by multiplication with appropriate Cartan matrix.
            Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='fw' , To='mx' )
            Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling(Decomposition_Style='Coarse')
            mixed = self.Base_Space().Basis('mx')
            Summands = []
            for Sym_HW , Sym_Mult in Get_Coefficients_And_Multiplicity_Of_Weyl_Character( Sym_Levi_Rep ) :
                # 2. Write the coefficients of the current considered weight in terms of the basis C (fw's)
                C_Coefficients = vector( QQ , n*[ 0 ] )
                C_Coefficients += sum([ Sym_HW[ Relabel-1 ] * e[Node] for Node , Relabel in Relabelling.items() ])
                # 3. Transform the coefficients from basis C (fw's) to B (mixed)
                B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
                # 4. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
                Sym_Levi_HW = 0*mixed[1]
                Sym_Levi_HW += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])

                Summands += [ ( self.__class__( Base_Space=self.Base_Space() , Highest_Weight=Sym_Twist_HW+Sym_Levi_HW ) , Sym_Mult*WCR(1) ) ]

            Result = Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=Summands )
            if Result.Is_Irreducible() : return Result[0]
            else                       : return Result


    def Weights ( self ) -> Iterator[ tuple[ "Weight" , int ] ] :
        """Returns the weights of ```self``` with are covered by the defining highest weight."""

        # Get all weights cover by the highest weight supported over the twisting part (excluded nodes)
        Weight_Twist = self.Highest_Weight_Supported_Over( Nodes=self.Base_Space().Parabolic_Subgroup().Excluded_Nodes() )

        # Get all weights cover by the highest weight supported over the Levi part (included nodes)
        Rep_Levi = self.Representation_Supported_Over_Levi_Part(Decomposition_Style='Coarse')
        Relabelling = self.Base_Space().Parabolic_Subgroup().Relabelling( Decomposition_Style='Coarse' )
        fw = self.Base_Space().Basis('fundamental weights')
        n = self.Base_Space().Parent_Group().Cartan_Degree()
        e = { i : vector( QQ , (i-1)*[ 0 ] + [ 1 ] + (n-i)*[ 0 ] ) for i in [ 1 .. n ] }
            # ``e`` is Standard basis to write coefficients of weights in handsome vector-notation and
            # to apply base changes use by multiplication with appropriate Cartan matrix.
        Cartan_Matrix_From_C_To_B = self.Base_Space().Cartan_Matrix( From='fw' , To='mx' )
        mixed = self.Base_Space().Basis('mx')
        for Weight_Levi_1 , Multiplicity in Rep_Levi.weight_multiplicities().items() :
            Weight_Levi_2 = self.Base_Space().Null_Weight()
            for Node , Relabel in Relabelling.items() :
                Weight_Levi_2 += Weight_Levi_1.to_weight_space().coefficient( Relabel ) * fw[Node]
            # In the remaining part, we merge the separate weights covered by the highest weights of both cases,
            # namely over both the twisting part (excluded nodes) and the Levi part (included nodes)
            # 1. Write the coefficients of the current considered weight in terms of the basis C (fw's)
            C_Coefficients = vector( QQ , n*[0] )
            C_Coefficients += sum([ Coefficient * e[Node] for Node , Coefficient in Weight_Levi_2 ])
            # 2. Transform the coefficients from basis C (fw's) to B (mixed)
            B_Coefficients = Cartan_Matrix_From_C_To_B * C_Coefficients
            # 3. Collect all coefficients supported over the included nodes (Note: Basis A and B coincide over the included nodes.)
            Weight_Levi = 0 * mixed[1]
            Weight_Levi += sum( [  Coefficient*mixed[Node] for Node , Coefficient in enumerate( list(B_Coefficients) , start=1 ) if Node in self.Base_Space().Included_Nodes() ])
            # 4. Form the weight
            Weight = Weight_Levi + Weight_Twist

            yield Weight , Multiplicity


In [30]:
class Direct_Sum_Of_Equivariant_Vector_Bundles ( Direct_Sum_Of_Algebraic_Structures , Equivariant_Vector_Bundle ) :

    def __contains__ ( self , other ) -> bool :
        if isinstance( other , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) :
            if (self-other).Negative_Part() == self.Base_Space().Zero_Vector_Bundle() : return True
            else                                                                      : return False

        else :
            return False


    def __init__( self , Base_Space:Irreducible_Homogeneous_Variety ,
                         Summands:list[ ( Irreducible_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundles" , "Weyl_Character" ) or \
                                        Irreducible_Equivariant_Vector_Bundle or \
                                        "Extension_Of_Equivariant_Vector_Bundles" \
                                      ]
                ) -> None :
        """
        Initialize a direct sum of equivariant vector bundles over an irreducible homogeneous variety.

        INPUT:
        - ``Components`` -- list of tuples consisting of irreducible equivariant vector bundles (and multiplicities);

        OUTPUT: None.
        """
        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``Base_Space`` need to be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space

        WCR = self._Base_Space.Parent_Group().Weyl_Character_Ring()

        assert isinstance( Summands , list ) , \
               TypeError('The input for ``Summands`` need to be a list.')

        self._Summands = []
        for Summand_Counter , Summand in enumerate( Summands , start=1 ) :
            if   isinstance( Summand , tuple ) :
                assert len(Summand) == 2 , \
                       ValueError('If the '+str(Summand_Counter)+'-th summand is given as tuple, it must be of length 2.')

            elif isinstance( Summand , ( Irreducible_Equivariant_Vector_Bundle , Extension_Of_Equivariant_Vector_Bundles ) ) :
                Summand = ( Summand , WCR(1) )

            else :
                raise TypeError('The '+str(Summand_Counter)+'-th summand need to be a tuple consisting of an irreducible equivariant vector bundle and a Weyl Character or' \
                                + 'or it need to be just an irreducible equivariant vector bundle.'
                               )

            VB , Mult = Summand
            assert isinstance( VB , ( Irreducible_Equivariant_Vector_Bundle , Extension_Of_Equivariant_Vector_Bundles ) ), \
                   TypeError('The '+str(Summand_Counter)+'-th summand need to contain an irreducible equivairant vector bundle or an extension as first element.')

            assert self._Base_Space == VB.Base_Space() , \
                   ValueError('The base space of the '+str(Summand_Counter)+'-th summand does not coincide with the input for ``Base_Space``.')

            assert Mult in WCR , \
                   ValueError('The multiplicity of the '+str(Summand_Counter)+'-th summnad is not an element of the Weyl character ring ``'+str(WCR)+'``.')

            self._Summands += [ ( VB , Mult ) ]

            self.Tidy_Up()


    def __iter__ ( self ) -> Iterator[ "Irreducible_Equivariant_Equivariant" or "Direct_Sum_Of_Equivariant_Vector_Bundles" ] :
        """Returns a single datum or slices of data."""
        for VB , Mult in super().__iter__() :
            Summand = self.__class__( Base_Space=self.Base_Space() , Summands=[ ( VB , Mult ) ] )
            if Summand.Is_Irreducible() : yield Summand[0]
            else : yield Summand


    def __getitem__ ( self , Input:int or slice ) -> "Irreducible_Equivariant_Equivariant" or list[ "Irreducible_Equivariant_Equivariant" ] :
        """Returns a single datum or slices of data."""
        Result = super().__getitem__( Input )
        if   Result == None :
            return self.Base_Space().Zero_Vector_Bundle()

        elif isinstance( Result , tuple ) :
            VB , Mult = Result
            Result = self.__class__( Base_Space=self.Base_Space() , Summands=[ ( VB , Mult ) ] )

            if Result.Is_Irreducible() :
                VB = Result.Equivariant_Vector_Bundles()[0]
                return VB
            else :
                return Result

        else :
            return Result


    def __mul__ ( self , \
                  other:Integer or "Weyl_Character" or Irreducible_Equivariant_Vector_Bundle or "Direct_Sum_Of_Equivariant_Vector_Bundle" or "Extension_Of_Equivariant_Vector_Bundles" \
                ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" or "Extension_Of_Equivariant_Vector_Bundles" :
        """Returns the tensor product of ``self`` with ``other``."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

        if   isinstance( other , Integer ) :
            return self * (other*WCR(1))

        elif isinstance( other , type(WCR(0)) ) :
            return self * Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=self.Base_Space() , Summands=[ ( self.Base_Space().calO() , other ) ] )

        elif isinstance( other , Irreducible_Equivariant_Vector_Bundle ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            return self * other.SemiSimplification()

        elif isinstance( other , self.__class__ ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            Product = self.Base_Space().Zero_Vector_Bundle()
            for Vector_Bundle_1 , Multiplicity_1 in self.Summands() :
                for Vector_Bundle_2 , Multiplicity_2 in other.Summands() :
                    for Weyl_Character_HW , Weyl_Character_Multiplicity in Multiplicity_1 * Multiplicity_2 :
                        for VB , Mult in ( Vector_Bundle_1 * Vector_Bundle_2 ).SemiSimplification().Summands() :
                            Product += self.__class__( Base_Space=self.Base_Space() , Summands=[ ( VB , Mult * Weyl_Character_Multiplicity * WCR(Weyl_Character_HW) ) ] )

            if isinstance( Product , self.__class__ ) : Product.Tidy_Up()
            return Product

        elif isinstance( other , Extension_Of_Equivariant_Vector_Bundles ) :
            assert self.Base_Space() == other.Base_Space() , \
                   ValueError( 'The base spaces of ``self`` and ``other`` do not coincide.' )

            return other.__class__( Base_Space=other.Base_Space() , Quotient=self*other.Quotient() , Subobject=self*other.Subobject() )

        else :
            raise ValueError( 'The input for ``other`` must be an equivariant vector bundle over an irreducible homogeneous variety or' \
                              + 'the extension of those ones.'
                            )


    def __neg__ ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        return self.__class__( Base_Space=self.Base_Space() , Summands=[ ( VB , -Mult ) for VB , Mult in self.Summands() ] )


    def __repr__ ( self ) -> ( Irreducible_Homogeneous_Variety , list[ ( Irreducible_Equivariant_Vector_Bundle or "Equivariant_Extension_Of_Vector_Bundles" , "Weyl_Character" )  ] ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Summands()


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Strings = []
        for Vector_Bundle , Multiplicity in self.Summands() :
            if isinstance( Vector_Bundle , Extension_Of_Equivariant_Vector_Bundles ) : str_VB = str(Vector_Bundle.Abbreviation())
            else                                                                     : str_VB = str(Vector_Bundle)

            if   Multiplicity == WCR(1) : str_Mult = ''
            else :
                if len(Multiplicity) == 1 : str_Mult = str(Multiplicity)+' * '
                else                      : str_Mult = '( '+str(Multiplicity)+' ) * '

            Strings += [ str_Mult+str_VB ]

        if len(Strings) == 0 : return '0'
        else                 : return ' + '.join(Strings)


    def Cohomology ( self , *p:int ) -> dict[ int : "Weyl_Character" ] :
        """Returns the cohomology."""
        Cohomology = {}
        for Summand , Multiplicity in self.Summands() :
            for Degree , Cohomology_Group in Summand.Cohomology().items() :
                if not Degree in Cohomology.keys() : Cohomology.update({ Degree : 0 })
                Cohomology[Degree] += Multiplicity*Cohomology_Group

        if   len(p) == 0 : return Cohomology

        elif len(p) == 1 :
            if p in Cohomology.keys() : return { p : Cohomology[p] }
            else : return dict({})

        else :
            raise ValueError('The input for ``p`` is of inappropriate length.')


    def Difference( self , other ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the difference of ``self`` with respect to ``other``."""
        if isinstance( other , self.__class__ ) :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            if isinstance( other , [ Irreducible_Equivariant_Vector_Bundle , Extension_Of_Equivariant_Vector_Bundles ] ) :
                other = self.__class__( Base_Space=other.Base_Space() , Summands=[ other ] )

            Remaining_Summands_Of_self  = []
            Remaining_Summands_Of_other = []
            for other_VB , other_Mult in other.Summands() :
                for self_VB , self_Mult in self.Summands() :
                    other_VB_Is_Found_In_self = False
                    if other_VB == self_VB :
                        other_VB_Is_Found_In_self = True
                        for hw , mult in self_Mult-other_Mult :
                            if 0 <= mult : Remaining_Summands_Of_self  += [ ( self_VB ,      mult*WCR(hw) ) ]
                            else         : Remaining_Summands_Of_other += [ ( self_VB , abs(mult)*WCR(hw) ) ]
                        break
                if not other_VB_Is_Found_In_self : Remaining_Summands_Of_other += [ ( other_VB , other_Mult ) ]

            return self.__class__( Base_Space= self.Base_Space() , Summands=Remaining_Summands_Of_self  ) , \
                   self.__class__( Base_Space=other.Base_Space() , Summands=Remaining_Summands_Of_other )
        else :
            raise ValueError('The input for ``other`` need to be an equivariant vector bundle.')


    def Dual ( self ) -> "Direct_Sum_of_Equivariant_Vector_Bundles" :
        """Returns the dual of ``self``."""
        return self.__class__( Base_Space=self.Base_Space() ,
                               Summands=[ ( Vector_Bundle.Dual() , Multiplicity ) for Vector_Bundle , Multiplicity in self.Summands() ]
                             )


    def Equivariant_Vector_Bundles ( self ) -> list[ Irreducible_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundles" ] :
        """Returns the attribute ``_Irreducible_Equivariant_Vector_Bundles``."""
        return [ Vector_Bundle for Vector_Bundle , Multiplicity in self.Summands() ]


    def Exterior_Power ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """
        Returns the ``e``-th exterior power of ``self``.

        We need to compute wedge^p( rmV^{ omega_1 } * calU^{ omega_2 } ) for a single summand.
        It holds
            wedge^p( rmV^{ omega_1 } * calU^{ omega_2 } )
                = alpha_p ( E_1 , ... , E_p  , F_1 , ... , F_p )
        with
            E_i = wedge^i rmV^{ omega_1 } * calO
        and
            F_j = wedge^j calU^{ omega_2 }.
        \alpha_p is the polynomial in 2p indeterminates defined by the formula
            sum_p alpha_p * T^p
                = prod_{ i,j >= 1 } ( 1 + X_i Y_j T ) =: beta
        where E_i is an elementary symmetric polynomial in the X_i's and F_j is such one in the Y_j's.

        https://concretenonsense.wordpress.com/2009/07/23/lambda-rings/ (Accessed:2023-02-08)
        """
        assert p in ZZ , \
               ValueError('The exponent ``e`` must be an integer.')

        if   p == 0 :
            return self.Base_Space().calO()

        elif p == 1 :
            return self

        elif p in [ 2 .. self.Rank() ] :

            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

            def Subroutine_For_Exterior_Power_Of_Summand ( Subexponent_For_Summand , Summand ) :
                Vector_Bundle , Multiplicity = Summand
                if   Vector_Bundle == self.Base_Space().calO() :
                    return self.__class__( Base_Space=self.Base_Space() , Summands=[ ( Vector_Bundle , Multiplicity.exterior_power(Subexponent_For_Summand) ) ] )

                elif Multiplicity == WCR(1) :
                    return ( Vector_Bundle.Exterior_Power(Subexponent_For_Summand) ).SemiSimplification()

                else :
                    #A = PolynomialRing( ZZ  , [ 'E'+str(Counter) for Counter in [ 1 .. p ] ] + [ 'F'+str(Counter) for Counter in [ 1 .. p ] ] )
                    #E = A.gens()[ : p ]
                    #F = A.gens()[ p : ]

                    B_X = PolynomialRing( ZZ  , [ 'X'+str(Counter) for Counter in [ 1 .. Subexponent_For_Summand ] ] ) ; X = B_X.gens()
                    B_Y = PolynomialRing( B_X , [ 'Y'+str(Counter) for Counter in [ 1 .. Subexponent_For_Summand ] ] ) ; Y = B_Y.gens()    ; B0 = B_Y
                    B   = PolynomialRing( B0  , [ 'T' ] )                                                              ; T = B.gens()[ 0 ]

                    e = SymmetricFunctions(ZZ).elementary()
                    f = SymmetricFunctions(B_X).elementary()

                    if p == 0 : beta = [ 1 ]
                    else : beta = prod([ 1 + X[Counter1-1]*Y[Counter2-1] * T for Counter1 in [ 1 .. Subexponent_For_Summand ] for Counter2 in [ 1 .. Subexponent_For_Summand ] ]).coefficients()
                        # The compuation for beta are quite expensive.
                        # Hence, we implemented it to the case where they are needed for sure.
                        # However, by the current implementation, the computation is repeated for each step in the for-loop where this case is called.
                        # A better approach would be to do the computation with `p` instead of `Subexponent_For_Summand` if this case is called the first time and then store the values globally
                        # such that any later evaluation of this case can fall back to the data.

                    alpha = self.Base_Space().Zero_Vector_Bundle()

                    if   Subexponent_For_Summand == 0 : return self.Base_Space().calO()
                    elif Subexponent_For_Summand == 1 : return self.__class__( Base_Space=self.Base_Space() , Summnads=[ Summand ] )

                    for Degree1 , Coefficient1 in f.from_polynomial( beta[Subexponent_For_Summand] ).monomial_coefficients().items() :

                        #Factor1 = prod([ F[Counter-1] for Counter in Degree1 ])
                        Factor_VB = self.Base_Space().calO()
                        for Counter in Degree1 :
                            Factor_VB *= ( Vector_Bundle.Exterior_Power(Counter) ).SemiSimplification()

                        Factor_Mult = WCR(0)
                        for Degree2 , Coefficient2 in e.from_polynomial( Coefficient1 ).monomial_coefficients().items() :
                            #Factor2 += Coefficient2 * prod([ E[Counter-1] for Counter in Degree2 ])
                            Factor_Mult += Coefficient2 * prod([ Multiplicity.exterior_power(Counter) for Counter in Degree2 ])

                        alpha += Factor_VB * Factor_Mult

                    return alpha

            Result = self.Base_Space().Zero_Vector_Bundle()
            for Partition_For_Summand in IntegerListsLex( n=p , length=len(self) ) :
                New_Summand = self.Base_Space().calO()
                for Summand_Counter , Subexponent_For_Summand in enumerate( Partition_For_Summand ) :
                    New_Summand *= Subroutine_For_Exterior_Power_Of_Summand( Subexponent_For_Summand , self.Summands()[Summand_Counter] )
                Result += New_Summand
            return Result

        else : return self.Base_Space().Zero_Vector_Bundle()


    def Is_Irreducible ( self ) -> bool :
        """Tests if ``self`` is irreducible/ simple."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        return len(self) == 1 and self.Multiplicities()[0] == WCR(1)


    def Irreducible_Components ( self ) -> Iterator[ Irreducible_Equivariant_Vector_Bundle or "Direct_Sum_Of_Equivariant_Vector_Bundles" ] :
        """Returns the irreducible components of ``self``."""

        for Factor , Weyl_Character in self.Summands() :
            #Factor = self.__class__( Base_Space=self.Base_Space() , Summands=[ Vector_Bundle ] )

            Dict = {}
            for Weight , Multiplicity in Weyl_Character.weight_multiplicities().items() :
                Weight = Weight.to_weight_space()
                if self.Base_Space().Is_Levi_Dominant( Weight ) :
                    rk = self.Base_Space().calU( Weight ).Rank()
                    if not rk in Dict.keys() : Dict.update({ rk : [] })
                    Dict[rk] += Multiplicity * [ Weight ]

            Rks = sorted(Dict.keys())
            Rks.reverse()

            for rk_1 in Rks :
                while 0 < len(Dict[rk_1]) :
                    Highest_Weight = Dict[rk_1][0]
                    Irreducible_Object = Irreducible_Equivariant_Vector_Bundle( Base_Space=self.Base_Space() , Highest_Weight=Highest_Weight )
                    for Summand in Irreducible_Object * Factor :
                        yield Summand

                    for Weight , Multiplicity in Irreducible_Object.Weights() :
                        if self.Base_Space().Is_Levi_Dominant( Weight ) :
                            for Counter in [ 1 .. Multiplicity ] :
                                rk_2 = self.Base_Space().calU( Weight ).Rank()
                                Dict[rk_2].remove(Weight)


    def Multiplicities ( self ) -> list[ "Weyl_Character" ] :
        """Returns the attribute ``_Multiplicities``."""
        return [ Multiplicity for Vector_Bundle , Multiplicity in self.Summands() ]


    def Negative_Part ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        return -( (-self).Positive_Part() )


    def Positive_Part ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Positive_Summands = []
        for VB , Weyl_Character in self.Summands() :
            Positive_Part_Of_Weyl_Character = WCR(0)
            for hw , mult in Weyl_Character :
                if 0 <= mult : Positive_Part_Of_Weyl_Character += mult*WCR(hw)
            Positive_Summands += [ ( VB , Positive_Part_Of_Weyl_Character ) ]
        return self.__class__( Base_Space=self.Base_Space() , Summands=Positive_Summands )


    def PullBack ( self , Image:Irreducible_Homogeneous_Variety , Dictionary:dict ) -> Equivariant_Vector_Bundle :
        """Returns the semi-simplification of the pull back of ``self`` to ``Image``."""
        assert isinstance( Image , Irreducible_Homogeneous_Variety ) , \
               'The input for ``Image`` need Image be an irreducible homogeneous variety.'
        SemiSimplification = Image.Zero_Vector_Bundle()
        for Summand in self.Irreducible_Components() :
            SemiSimplification += Summand.PullBack( Image , Dictionary )
        return SemiSimplification


    def Rank ( self ) -> int :
        """Returns the rank of ``self``."""
        return sum([ Vector_Bundle.Rank() * Multiplicity.degree() for Vector_Bundle , Multiplicity in self.Summands() ])


    def SemiSimplification ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns ``self`` as direct sum."""
        SemiSimplification = self.Base_Space().Zero_Vector_Bundle()
        for Summand in self :
            if isinstance( Summand , Extension_Of_Equivariant_Vector_Bundles ) : SemiSimplification += Summand.SemiSimplification()
            else                                                               : SemiSimplification += Summand
        return SemiSimplification


    def Symmetric_Power ( self , p:int ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """
        Returns the ``p``-th symmetric power of ``self``.

        ..ToDo: Implement the case S^p( Mult * VB ) where Mult != WCR(1) and VB != 0.
        """
        assert p in Integers() , 'The exponent ``e`` must be an integer.'

        if   p <  0 : return self.Base_Space().Zero_Vector_Bundle()
        elif p == 0 : return self.Base_Space().calO()
        elif p == 1 : return self
        else :

            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()

            def Subroutine_For_Symmetric_Power_Of_Summand ( Subexponent_For_Summand , Summand ) :
                Vector_Bundle , Multiplicity = Summand
                if   Vector_Bundle == self.Base_Space().calO() :
                    return self.__class__( Base_Space=self.Base_Space() , Summands=[ ( Vector_Bundle , Multiplicity.symmetric_power(Subexponent_For_Summand) ) ] )

                elif Multiplicity == WCR(1) :
                    return ( Vector_Bundle.Symmetric_Power(Subexponent_For_Summand) ).SemiSimplification()

                else :
                    raise ValueError('The case S^p( Mult * VB ) where both Mult and VB are non-trivial is NOT yet implemented.')

            Result = self.Base_Space().Zero_Vector_Bundle()
            for Partition_For_Summand in IntegerListsLex( n=p , length=len(self) ) :
                New_Summand = self.Base_Space().calO()
                New_Summand *= prod([ Subroutine_For_Symmetric_Power_Of_Summand( Subexponent_For_Summand , self.Summands()[Summand_Counter] )
                                      for Summand_Counter , Subexponent_For_Summand in enumerate( Partition_For_Summand )
                                    ])
                Result += New_Summand
            return Result


    def Tidy_Up ( self ) -> None :
        """Returns ``self`` after merging components with the same vector bundle and dropping zero components."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        Summands = []
        for Summand_Counter_1 , ( Vector_Bundle_1 , Multiplicity_1 ) in enumerate( self.Summands() ) :
            if Multiplicity_1 == 0 :
                pass
            else :
                Summand_Is_Implemented_After_Tidying_Up = False
                for Summand_Counter_2 , ( Vector_Bundle_2 , Multiplicity_2 ) in enumerate( Summands ) :
                    if Vector_Bundle_1 == Vector_Bundle_2 :
                        Summands[Summand_Counter_2] = ( Vector_Bundle_2 , Multiplicity_2+Multiplicity_1 )
                        Summand_Is_Implemented_After_Tidying_Up = True
                        break

                if not Summand_Is_Implemented_After_Tidying_Up :
                    Summands += [ ( Vector_Bundle_1 , Multiplicity_1 ) ]

        self._Summands = Summands


In [17]:
class Extension_Of_Equivariant_Vector_Bundles ( Extension_Of_Algebraic_Structures , Equivariant_Vector_Bundle ) :

    def __init__ ( self , Base_Space:Irreducible_Homogeneous_Variety , \
                   Quotient:Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundle" , \
                   Subobject:Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundle" , \
                 ) -> None :
        assert isinstance( Quotient , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) , \
               TypeError('The quotient must be a vector bundle over an irreducible homogeneous variety.')
        assert isinstance( Subobject , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) , \
               TypeError('The subobject must be a vector bundle over an irreducible homogeneous variety.')

        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``Base_Space`` must be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space
        WCR = self._Base_Space.Parent_Group().Weyl_Character_Ring()

        assert Base_Space == Quotient.Base_Space() , \
               ValueError('The base space of ``Quotient`` does not coincide with the given base space.')
        assert Base_Space == Subobject.Base_Space() , \
               ValueError('The base space of ``Subobject`` does not coincide with the given base space.')

        self._Depths = []
        for Degree , Weyl_Character in Quotient.EXT( Subobject ).items() :
            for Highest_Weight , Multiplicity in Weyl_Character :
                if WCR( Highest_Weight ) == WCR(1) and 0 < Multiplicity : self._Depths += [ Degree ]
        if not 0 < len(self._Depths) :
            raise Exception(   'There are no trivial representations for an equivariant extension in EXT(Quotient,Subobject).' + '\n' \
                             + 'Thus, the given data is not a candidate for an equivariant extension.'
                           )
        self._Subobject = Subobject
        self._Quotient = Quotient


    def __mul__ ( self , \
                  other:Integer or "Weyl_Character" or Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundles" \
                ) -> "Extension_Of_Equivariant_Vector_Bundle" :
        """Returns the tensor product of ``self`` with ``other``."""
        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        assert isinstance( other , ( Integer , type(WCR(0)) , Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) , \
               TypeError('The input for ``other`` of an inappropriate type.')
        return self.__class__( Base_Space=self.Base_Space() , Quotient=self.Quotient()*other , Subobject=self.Subobject()*other )


    def __neg__ ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        Base_Space=self.Base_Space()
        WCR = Base_Space.Parent_Group().Weyl_Character_Ring()
        Summands = [ ( self , -WCR(1) ) ]
        return Direct_Sum_Of_Equivariant_Vector_Bundles( Base_Space=Base_Space , Summands=Summands )


    def __repr__ ( self ) -> ( Irreducible_Homogeneous_Variety , \
                               Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundle" , \
                               Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundle or "Extension_Of_Equivariant_Vector_Bundle"
                             ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self._Base_Space , self.Quotient() , self.Subobject()


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        Depths = self.Depths()

        if   Depths == [ 1 ]  :
            String = 'Equivariant extension'
        elif len(Depths) == 1 :
            Depth = Depths[0]
            String = 'Equivariant '+str(Depth)+'-extension'
        else                      :
            String = 'Equivariant '+str(Depths)+'-extension'

        if isinstance( self.Quotient() , self.__class__ ) : Quotient = str(self.Quotient().Abbreviation())
        else                                              : Quotient = str(self.Quotient())

        if isinstance( self.Subobject() , self.__class__ ) : Subobject = str(self.Subobject().Abbreviation())
        else                                               : Subobject = str(self.Subobject())

        return String+' of ' + Quotient + ' by ' + Subobject


    def Abbreviation ( self ) -> str :
        """Returns a description of ``self`` in [Quotient,Subobject]-style."""
        List = []
        for Summand in [ self.Quotient() , self.Subobject() ] :
            if isinstance( Summand , self.__class__ ) : List += [ str( Summand.Abbreviation() ) ]
            else                                      : List += [ str( Summand ) ]
        return '[ ' + ' , '.join(List) + ' ]'


    def Base_Space ( self ) -> Irreducible_Homogeneous_Variety :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    def Cohomology ( self , *p ) -> dict[ int : "Weyl_Character" ]:
        """Returns the cohomology of ``self``."""
        return self.SemiSimplification().Cohomology( *p )


    def Depths ( self ) -> list[int] :
        """Returns the attribute ``depth`` -- i.e. EXT^d( A , B ) contains a trivial representation."""
        return self._Depths


    def Determinant_Bundle ( self ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the determinant line bundle of self (= rk-th top exterior power of ``self``)."""
        return self.Subobject().Determinant_Bundle() * self.Quotient().Determinant_Bundle()


    def Dual ( self ) -> "Extension_Of_Equivariant_Vector_Bundles" :
        """Returns the dual of ``self``."""
        return self.__class__( Base_Space=self.Base_Space() , Quotient=self.Subobject().Dual() , Subobject=self.Quotient().Dual() )


    def Exact_Sequences ( self , Index=0 , Tab='' ) -> Iterator[ 'str' ] :
        """Returns a description of ``self`` in terms of exact sequences."""
        assert Index in Integers() , 'The input for ``Index`` must be an integer.'
        assert isinstance( Tab , str ) , 'The input for ``Tab`` must be a string.'

        Object = 'E'+str(Index)

        if isinstance( self.Quotient() , self.__class__ ) :
            Quotient_Is_Extension = True
            Quotient_Index = Index+1
            Quotient = 'E'+str(Quotient_Index)
        else :
            Quotient_Is_Extension = False
            Quotient = str( self.Quotient() )

        if isinstance( self.Subobject() , self.__class__ ) :
            Subobject_Is_Extension = True
            if Quotient_Is_Extension : Subobject_Index = Index+2
            else : Subobject_Index = Index+1
            Subobject = 'E'+str(Subobject_Index)
        else :
            Subobject_Is_Extension = False
            Subobject = str( self.Subobject() )

        Depths = self.Depths()
        if isinstance( Depths , list ) : Depth = min(Depths)

        if   Depth == 0 : yield Quotient+' ---> '+Subobject
        elif Depth == 1 : yield '0 ---> '+Subobject+' ---> '+Object+' ---> '+Quotient+' ---> 0'
        elif Depth == 2 :  yield '0 ---> '+Subobject+' ---> '+Object+'_2'+' ---> '+Object+'_1'+' ---> '+Quotient+' ---> 0'
        elif 2 <  Depth :  yield '0 ---> '+Subobject+' ---> '+Object+'_'+str(Depth)+' ---> ... ---> '+Object+'_1'+' ---> '+Quotient+' ---> 0'

        if Quotient_Is_Extension :
            for Seq in self.Quotient().Exact_Sequences( Quotient_Index , Tab=Tab ) :
                yield Tab + Seq
        if Subobject_Is_Extension :
            for Seq in self.Subobject().Exact_Sequences( Subobject_Index , Tab=Tab ) :
                yield Tab + Seq


    def Negative_Part ( self ) -> Direct_Sum_Of_Equivariant_Vector_Bundles :
        return self.Base_Space().Zero_Vector_Bundle()


    def Positive_Part ( self ) -> "Extension_Of_Equivariant_Vector_Bundle" :
        return self


    def Rank ( self ) -> int :
        """Returns the rank of ``self``."""
        return self.Quotient().Rank() + self.Subobject().Rank()


    def SemiSimplification ( self ) -> Direct_Sum_Of_Equivariant_Vector_Bundles :
        """Returns the semi-simplification of ``self``."""
        return self.Quotient().SemiSimplification() + self.Subobject().SemiSimplification()
